# Von Neumann Stability Examples

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import fd_tools as fd

## Introduction

This notebook demonstrates von Neumann stability analysis for the 1D advection equation

\begin{gather*}
u_t + au_x = 0,\; (a>0)\\
u(x,0) = u_0(x)\\
u(0,t) = g(t)
\end{gather*}

using a few different time-stepping schemes.  The spatial derivative is approximated by the finite difference scheme

$$
D_\tau = \tau D_- + (1-\tau) D_+
$$

where $\tau\in[0,1]$ is a parameter.

For given $k$, $\Delta x$ and $\theta=k\Delta x$, the $D_-$ and $D_+$ operators scale $e^{ikx}$ as
$$
D_-e^{ikx} = \frac{e^{ikx}-e^{ik(x-\Delta x)}}{\Delta x} = 
\frac{1}{\Delta x}\left(1-e^{-ik\Delta x}\right)e^{ikx} = \frac{\lambda_-(\theta)}{\Delta x}e^{ikx}
$$
and
$$
D_+e^{ikx} = \frac{e^{ik(x+\Delta x)}-e^{ikx}}{\Delta x} = 
\frac{1}{\Delta x}\left(e^{ik\Delta x}-1\right)e^{ikx} = \frac{\lambda_+(\theta)}{\Delta x}e^{ikx}.
$$

Thus, $D_\tau$ scales $e^{ikx}$ as

$$
D_\tau e^{ikx} = \frac{\tau\lambda_-(\theta)+(1-\tau)\lambda_+(\theta)}{\Delta x}e^{ikx}
= \frac{\lambda_\tau(k,\Delta x)}{\Delta x}e^{ikx}.
$$

In [ ]:
def lambda_minus(theta):
    return (1-np.exp(-1j*theta))

def lambda_plus(theta):
    return (np.exp(1j*theta)-1)

            
def lambda_tau(tau, theta):
    return tau*lambda_minus(theta) + (1-tau)*lambda_plus(theta)

## Forward Euler

The forward Euler method for this problem takes the form

$$
U^{j+1} = U^j - a\Delta t D_\tau U^j
$$

Substituting the ansatz $U^j=g^je^{ikx}$ yields
$$
g^{j+1}e^{ikx} = g^je^{ikx} - \frac{a\Delta t}{\Delta x}\lambda_\tau(\theta)g^je^{ikx}.
$$
Dividing by $g^je^{ikx}$ simplifies this to
$$
g = 1 - \frac{a\Delta t}{\Delta x}\lambda_\tau(\theta).
$$

As we (almost) saw in class, as theta varies, $g$ traces an ellipse in the complex plane.

In [ ]:
def fe_vn_demo():
    a = 1.8
    dx = 0.1
    tau = 0.9

    dt = (dx/a)*(2*tau-1)

    theta = np.linspace(0, 2*np.pi, 1000)

    g = 1 - (a*dt/dx)*lambda_tau(tau, theta)

    for i in range(2):
        plt.figure(figsize=(5,5))
        plt.plot(np.real(g), np.imag(g))
        plt.plot(np.cos(theta), np.sin(theta), '--')
        if i == 0:
            plt.axis('equal')
            plt.title('dx = {:.3e}, dt = {:.3e}'.format(dx, dt))
        else:
            plt.xlim([0.99, 1.0])
            plt.ylim([-0.15, 0.15])
    
fe_vn_demo()

## Trapezoidal method

The trapezoidal method takes the form

$$
U^{j+1} = U^j - \frac{a\Delta t}{2}\left[D_\tau U^{j+1} + D_\tau U^j\right]
$$

Substituting the ansatz $U^j=g^je^{ikx}$ yields
$$
g^{j+1}e^{ikx} = g^je^{ikx} - \frac{a\Delta t}{2\Delta x}\left[
\lambda_\tau(\theta)g^{j+1}e^{ikx} + \lambda_\tau(\theta)g^je^{ikx}
\right].
$$
Dividing by $g^je^{ikx}$ simplifies this to
$$
g = 1 - \frac{a\Delta t}{2\Delta x}\left[
\lambda_\tau(\theta)g + \lambda_\tau(\theta)
\right],
$$
which can be rewritten as
$$
g = \frac{1 - \frac{a\Delta t}{2\Delta x}\lambda_\tau(\theta)}
{1 + \frac{a\Delta t}{2\Delta x}\lambda_\tau(\theta)}.
$$

As theta varies, $g$ traces an interesting curve.

In [ ]:
def tpz_vn_demo():
    a = 1.8
    dx, dt = 0.1, 0.3
    tau = 0.7

    theta = np.linspace(0, 2*np.pi, 10000)

    g = (1 - (a*dt)/(2*dx)*lambda_tau(tau, theta))/(1 + (a*dt)/(2*dx)*lambda_tau(tau, theta))

    for i in range(2):
        plt.figure(figsize=(5,5))
        plt.plot(np.real(g), np.imag(g))
        plt.plot(np.cos(theta), np.sin(theta), '--')
        if i == 0:
            plt.axis('equal')
            plt.title('dx = {:.3e}, dt = {:.3e}'.format(dx, dt))
        else:
            plt.xlim([0.99, 1.0])
            plt.ylim([-0.15, 0.15])
    
tpz_vn_demo()

## RK2

The second-order Runge-Kutta method takes the form

\begin{align*}
    k_1 &= -a\Delta t D_{\tau} U^j\\
    k_2 &= -a\Delta t D_{\tau}\left(U^j + \frac{\Delta t}{2}k_1\right)\\
    U^{j+1} &= U^j + k_2
\end{align*}

Substituting the ansatz $U^j=g^je^{ikx}$ yields
\begin{align*}
    k_1 &= -\frac{a\Delta t}{\Delta x} \lambda_\tau(\theta)g^je^{ikx}\\
    k_2 &= -a\Delta t D_{\tau}\left(g^je^{ikx} - \frac{a\Delta t^2}{2\Delta x}\lambda_\tau(\theta)g^je^{ikx}\right)
    = \left(-\frac{a\Delta t}{\Delta x}\lambda_\tau(\theta) + \frac{a^2\Delta t^3}{2\Delta x^2}\lambda_\tau(\theta)^2\right)g^je^{ikx}\\
    g^{j+1}e^{ikx} &= \left(1-\frac{a\Delta t}{\Delta x}\lambda_\tau(\theta) + \frac{a^2\Delta t^3}{2\Delta x^2}\lambda_\tau(\theta)^2\right)g^je^{ikx}
\end{align*}
Dividing by $g^je^{ikx}$ simplifies this to
$$
g = 1-\frac{a\Delta t}{\Delta x}\lambda_\tau(\theta) + \frac{a^2\Delta t^3}{2\Delta x^2}\lambda_\tau(\theta)^2.
$$

In [ ]:
def rk2_vn_demo():
    a = 1.8
    dx, dt = 0.1, 0.02
    tau = 0.7

    theta = np.linspace(0, 2*np.pi, 10000)

    g = 1 - (a*dt/dx)*lambda_tau(tau, theta) + (a**2*dt**3)/(2*dx**2)*lambda_tau(tau, theta)**2

    for i in range(2):
        plt.figure(figsize=(5,5))
        plt.plot(np.real(g), np.imag(g))
        plt.plot(np.cos(theta), np.sin(theta), '--')
        if i == 0:
            plt.axis('equal')
            plt.title('dx = {:.3e}, dt = {:.3e}'.format(dx, dt))
        else:
            plt.xlim([0.99, 1.0])
            plt.ylim([-0.15, 0.15])

rk2_vn_demo()